In [1]:
# Установим Spark:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
# Установим переменные среды:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 23.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767581 sha256=bf3c75e6adb0dd44c7b9e5be86447c292a54c088afaf2324872f7b65a62be2b5
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [6]:
sc = spark.sparkContext

In [7]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/commit/1b3a359c2d995c353d7a4b579d4662f608f8885f/data/trips.csv

--2024-03-16 12:05:29--  https://git.ai.ssau.ru/tk/big_data/raw/commit/1b3a359c2d995c353d7a4b579d4662f608f8885f/data/trips.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80208831 (76M) [text/plain]
Saving to: ‘trips.csv’

trips.csv           100%[===================>]  76.49M  2.58MB/s    in 46s     

2024-03-16 12:06:15 (1.67 MB/s) - ‘trips.csv’ saved [80208831/80208831]



In [8]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/stations.csv

--2024-03-16 12:06:15--  https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/stations.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5647 (5.5K) [text/plain]
Saving to: ‘stations.csv’

stations.csv        100%[===================>]   5.51K  --.-KB/s    in 0s      

2024-03-16 12:06:17 (52.7 MB/s) - ‘stations.csv’ saved [5647/5647]



In [9]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz  stations.csv  trips.csv


In [10]:
tripsData = sc.textFile('trips.csv')
tripsHeader = tripsData.first()
trips = tripsData.filter(lambda row: row != tripsHeader).map(lambda row: row.split(',', -1))

In [11]:
tripsData.take(5)

['id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code',
 '4576,63,,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060']

In [12]:
trips.take(2)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [13]:
stationsData = sc.textFile('stations.csv')
stationsHeader = stationsData.first()
stations = stationsData.filter(lambda row: row != stationsHeader).map(lambda row: row.split(',', -1))

In [14]:
stationsData.take(5)

['id,name,lat,long,dock_count,city,installation_date',
 '2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013']

In [15]:
stations.take(2)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013']]

In [16]:
list(enumerate(tripsHeader.split(',')))

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [17]:
list(enumerate(stationsHeader.split(',')))

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

In [18]:
stationsIndexed = stations.keyBy(lambda x: x[0])
stationsIndexed.take(2)

[('2',
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 ('3',
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013'])]

In [19]:
tripsByStartTerminals = trips.keyBy(lambda x: x[4])
tripsByEndTerminals = trips.keyBy(lambda x: x[7])

In [20]:
tripsByStartTerminals.take(2)

[('66',
  ['4576',
   '63',
   '',
   'South Van Ness at Market',
   '66',
   '8/29/2013 14:14',
   'South Van Ness at Market',
   '66',
   '520',
   'Subscriber',
   '94127']),
 ('10',
  ['4607',
   '',
   '8/29/2013 14:42',
   'San Jose City Hall',
   '10',
   '8/29/2013 14:43',
   'San Jose City Hall',
   '10',
   '661',
   'Subscriber',
   '95138'])]

In [21]:
tripsByEndTerminals.take(2)

[('66',
  ['4576',
   '63',
   '',
   'South Van Ness at Market',
   '66',
   '8/29/2013 14:14',
   'South Van Ness at Market',
   '66',
   '520',
   'Subscriber',
   '94127']),
 ('10',
  ['4607',
   '',
   '8/29/2013 14:42',
   'San Jose City Hall',
   '10',
   '8/29/2013 14:43',
   'San Jose City Hall',
   '10',
   '661',
   'Subscriber',
   '95138'])]

In [22]:
startTrips = stationsIndexed.join(tripsByStartTerminals)
endTrips = stationsIndexed.join(tripsByEndTerminals)

In [23]:
print(startTrips.toDebugString().decode('utf-8'))

(5) PythonRDD[27] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[19] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[18] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[17] at join at <ipython-input-22-d9b5f12a0e8e>:1 []
    |  PythonRDD[16] at join at <ipython-input-22-d9b5f12a0e8e>:1 []
    |  UnionRDD[15] at union at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[13] at RDD at PythonRDD.scala:53 []
    |  stations.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []
    |  stations.csv HadoopRDD[5] at textFile at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[14] at RDD at PythonRDD.scala:53 []
    |  trips.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  trips.csv HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [24]:
print(endTrips.toDebugString().decode('utf-8'))

(5) PythonRDD[28] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[26] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[25] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[24] at join at <ipython-input-22-d9b5f12a0e8e>:2 []
    |  PythonRDD[23] at join at <ipython-input-22-d9b5f12a0e8e>:2 []
    |  UnionRDD[22] at union at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[20] at RDD at PythonRDD.scala:53 []
    |  stations.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []
    |  stations.csv HadoopRDD[5] at textFile at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[21] at RDD at PythonRDD.scala:53 []
    |  trips.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  trips.csv HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [25]:
startTrips.count()

669959

In [26]:
endTrips.count()

669959

In [27]:
from typing import NamedTuple
from datetime import datetime

In [28]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str

    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

In [29]:
stationsInternal = stations.mapPartitions(initStation)
stationsInternal.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

In [80]:
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str

    for trip in trips:
        try:
            yield Trip(
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = int(trip[7]),
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            )
        except:
            pass

In [81]:
tripsInternal = trips.mapPartitions(initTrip)
tripsInternal.take(5)

[Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id=27, bike_id=48, subscription_type='Subscriber', zip_code='97214'),
 Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id=10, bike_id=26, subscription_type='Subscriber', zip_code='95060'),
 Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id=67, bike_id=319, subscription_type='Subscriber', zip_code='94103'),
 Trip(trip_id=4927, duration=103, start_d

**Для каждой стартовой станции найдем среднее время поездки.**

In [32]:
import numpy as np

In [33]:
tripsByStartStation = tripsInternal.keyBy(lambda x: x.start_station_name)
tripsByStartStation.take(3)

[('Mountain View City Hall',
  Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214')),
 ('San Jose City Hall',
  Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id='10', bike_id=26, subscription_type='Subscriber', zip_code='95060')),
 ('South Van Ness at Market',
  Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id='67', bike_id=319

In [34]:
avgDurationByStartStation = tripsByStartStation.mapValues(lambda trip: trip.duration).groupByKey().mapValues(lambda durations: np.mean(list(durations)).round(4))
avgDurationByStartStation.top(10, key=lambda x: x[1])

[('University and Emerson', 7090.2394),
 ('California Ave Caltrain Station', 4628.0058),
 ('Redwood City Public Library', 4579.2347),
 ('Park at Olive', 4438.1613),
 ('San Jose Civic Center', 4208.0169),
 ('Rengstorff Avenue / California Street', 4174.0824),
 ('Redwood City Medical Center', 3959.492),
 ('Palo Alto Caltrain Station', 3210.649),
 ('San Mateo County Center', 2716.77),
 ('Broadway at Main', 2481.2537)]

**Попробуем решить задачу с помощью aggregateByKey.**

In [105]:
? tripsByStartStation.aggregateByKey

In [36]:
def seqFunc(acc, duration):
    sum_durations, count = acc
    return (sum_durations + duration, count + 1)

def combFunc(acc1, acc2):
    sum_durations1, count1 = acc1
    sum_durations2, count2 = acc2
    return (sum_durations1 + sum_durations2, count1 + count2)

def calcMean(acc):
    sum_durations, count = acc
    return round(sum_durations / count, 4)

avgDurationByStartStation2 = tripsByStartStation.mapValues(lambda trip: trip.duration).aggregateByKey((0, 0), seqFunc, combFunc).mapValues(lambda acc: calcMean(acc))
avgDurationByStartStation2.top(10, key=lambda x: x[1])

[('University and Emerson', 7090.2394),
 ('California Ave Caltrain Station', 4628.0058),
 ('Redwood City Public Library', 4579.2347),
 ('Park at Olive', 4438.1613),
 ('San Jose Civic Center', 4208.0169),
 ('Rengstorff Avenue / California Street', 4174.0824),
 ('Redwood City Medical Center', 3959.492),
 ('Palo Alto Caltrain Station', 3210.649),
 ('San Mateo County Center', 2716.77),
 ('Broadway at Main', 2481.2537)]

**Теперь найдем самую раннюю поездку для каждой станции.**

In [37]:
def findEarlyTrip(trips):
    min_trip = trips[0]
    for i in range(1, len(trips)):
        if min_trip.start_date > trips[i].start_date:
            min_trip = trips[i]
    return min_trip

earlyTripByStation = tripsByStartStation.groupByKey().mapValues(lambda trips: findEarlyTrip(list(trips)))
earlyTripByStation.take(5)

[('South Van Ness at Market',
  Trip(trip_id=4073, duration=1067, start_date=datetime.datetime(2013, 8, 29, 9, 24), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 9, 42), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=321, subscription_type='Subscriber', zip_code='94703')),
 ('Santa Clara at Almaden',
  Trip(trip_id=4500, duration=109, start_date=datetime.datetime(2013, 8, 29, 13, 25), start_station_name='Santa Clara at Almaden', start_station_id=4, end_date=datetime.datetime(2013, 8, 29, 13, 27), end_station_name='Adobe on Almaden', end_station_id='5', bike_id=679, subscription_type='Subscriber', zip_code='95112')),
 ('Clay at Battery',
  Trip(trip_id=4283, duration=1712, start_date=datetime.datetime(2013, 8, 29, 11, 55), start_station_name='Clay at Battery', start_station_id=41, end_date=datetime.datetime(2013, 8, 29, 12, 23), end_station_name='Harry Bridges Plaza (Ferry Building)',

**Попробуем сделать то же самое с помощью reduceByKey.**

In [38]:
earlyTripByStation2 = tripsByStartStation.reduceByKey(lambda x, y: x if x.start_date < y.start_date else y)
earlyTripByStation2.take(5)

[('South Van Ness at Market',
  Trip(trip_id=4074, duration=1131, start_date=datetime.datetime(2013, 8, 29, 9, 24), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 9, 43), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=317, subscription_type='Subscriber', zip_code='94115')),
 ('Santa Clara at Almaden',
  Trip(trip_id=4500, duration=109, start_date=datetime.datetime(2013, 8, 29, 13, 25), start_station_name='Santa Clara at Almaden', start_station_id=4, end_date=datetime.datetime(2013, 8, 29, 13, 27), end_station_name='Adobe on Almaden', end_station_id='5', bike_id=679, subscription_type='Subscriber', zip_code='95112')),
 ('Clay at Battery',
  Trip(trip_id=4283, duration=1712, start_date=datetime.datetime(2013, 8, 29, 11, 55), start_station_name='Clay at Battery', start_station_id=41, end_date=datetime.datetime(2013, 8, 29, 12, 23), end_station_name='Harry Bridges Plaza (Ferry Building)',

### Решим предложенные задачи.

#### 4. Найти количество велосипедов в системе.

In [39]:
bikes_count = tripsInternal.map(lambda trip: trip.bike_id).distinct().count()
print(f'Кол-во велосипедов в системе: {bikes_count}')

Кол-во велосипедов в системе: 700


#### 5. Найти пользователей, потративших на поездки более 3 часов.

In [40]:
users = tripsInternal.filter(lambda trip: trip.duration > 3 * 60 * 60) \
                     .map(lambda trip: trip.zip_code) \
                     .distinct() \
                     .filter(lambda user: user != '')
print(f'Пользователи, потратившие на поездки более 3 часов:')
print(f'{users.take(10)}')

Пользователи, потратившие на поездки более 3 часов:
['58553', '94301', '94039', '94133', '93726', '94123', '4517', '29200', '45322', '94080']


#### 1. Найти велосипед с макимальным временем пробега.

In [41]:
bike = tripsInternal.keyBy(lambda trip: trip.bike_id) \
                     .mapValues(lambda trip: trip.duration) \
                     .reduceByKey(lambda x, y: x + y) \
                     .max(lambda x: x[1])
print(f'Велосипед с максимальным временем пробега: {bike[0]}')

Велосипед с максимальным временем пробега: 535


#### 3. Найти путь велосипеда с максимальным временем пробега через станции.

In [84]:
way = tripsInternal.filter(lambda trip: trip.bike_id == bike[0]) \
                   .sortBy(lambda trip: trip.start_date) \
                   .map(lambda trip: (trip.start_station_name, trip.end_station_name))
print(f'Путь велосипеда через станции:')
print(way.take(5))

Путь велосипеда через станции:
[('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)'), ('San Francisco Caltrain (Townsend at 4th)', 'San Francisco Caltrain 2 (330 Townsend)'), ('San Francisco Caltrain 2 (330 Townsend)', 'Market at Sansome'), ('Market at Sansome', '2nd at South Park'), ('2nd at Townsend', 'Davis at Jackson')]


#### 2. Найти наибольшее геодезическое расстояние между станциями.

In [196]:
# 1 способ: Будем искать расстояния только между станциями каждого пути
from geopy.distance import geodesic as GD

stations = stationsInternal.collect()

def calcGeodesic(res):
    s = list(filter(lambda x: x.station_id in res, stations))
    s = list(map(lambda x: (x.lat, x.long), s))
    ask = GD(s[0], s[1]).km
    return res, round(ask, 4)

distance = tripsInternal.map(lambda trip: (trip.start_station_id, trip.end_station_id)) \
                        .filter(lambda trip: trip[0] != trip[1]) \
                        .distinct() \
                        .map(lambda x: calcGeodesic(x)) \
                        .max(lambda x: x[1])
print(f'Наибольшее геодезическое расстоние между станциями в км: {distance}')

Наибольшее геодезическое расстоние между станциями в км: ((3, 72), 68.0579)


In [197]:
# 2 способ: Будем искать расстояние между каждой парой станций, которые есть в системе
distance = stationsInternal.cartesian(stationsInternal) \
                           .filter(lambda pair: pair[0] < pair[1]) \
                           .map(lambda stations: ((stations[0].station_id, stations[1].station_id), ((stations[0].lat, stations[0].long), (stations[1].lat, stations[1].long)))) \
                           .mapValues(lambda x: round(GD(x[0], x[1]).km, 4)) \
                           .max(lambda x: x[1])
print(f'Наибольшее геодезическое расстоние между станциями в км: {distance}')

Наибольшее геодезическое расстоние между станциями в км: ((16, 60), 69.921)
